In [ ]:
!pip install torch
!pip install transformers[torch]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_shell.py:99 in system                      │
│                                                                                                  │
│    96 │   if pip_warn:                                                                           │
│    97 │     kwargs.update({'also_return_output': True})                                          │
│    98 │                                                                                          │
│ ❱  99 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│   100 │                                                                                          │
│   101 │   if pip_warn:                                                                           │
│   102 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:453 in _system_compat   │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:167 in _run_command     │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# load the saved model state dictionary
state_dict = torch.load("/content/drive/MyDrive/finetuned-gpt2-convai-main/model.pt")

# instantiate a new GPT2LMHeadModel using the same configuration as the trained model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# adjust the size of the model parameters to match the saved state dictionary
state_dict['transformer.wte.weight'] = state_dict['transformer.wte.weight'][:model.config.vocab_size, :]
state_dict['lm_head.weight'] = state_dict['lm_head.weight'][:model.config.vocab_size, :]

# load the state dictionary into the model
model.load_state_dict(state_dict)

# set the device to use for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# set the model to evaluation mode
model.eval()

def infer(inp):
    inp = "<You> "+inp+" \n<bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output


#getting the output from the inference model
print("Ask me anything : ")
while True:
  inp = input()
  if(inp.lower()=='stop'):
    print("Thank you have a good day !")
    break
  print(infer(inp))

Ask me anything : 
hello


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<You> hello 
<bot>:  Congratulations. This is my name opf your name
how are you doing?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<You> how are you doing? 
<bot>:  [creaking] Oh,
stop
Thank you have a good day !
